In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,ShuffleSplit,GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

In [31]:
hit = pd.read_csv("Hitters.csv")
df =hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[["League","Division","NewLeague"]])
y = df["Salary"]
x_ = df.drop(["Salary","League","Division","NewLeague"], axis = 1).astype("float64")
x = pd.concat([x_,dms[["League_N","Division_W","NewLeague_N"]]],axis=1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)


In [32]:
bag_model = BaggingRegressor(bootstrap_features= True)
bag_model.fit(x_train,y_train)

BaggingRegressor(bootstrap_features=True)

In [33]:
bag_model.n_estimators

10

In [34]:
bag_model.estimators_

[DecisionTreeRegressor(random_state=1216799593),
 DecisionTreeRegressor(random_state=492789392),
 DecisionTreeRegressor(random_state=1600675188),
 DecisionTreeRegressor(random_state=1534222142),
 DecisionTreeRegressor(random_state=113117812),
 DecisionTreeRegressor(random_state=1988517987),
 DecisionTreeRegressor(random_state=1860101405),
 DecisionTreeRegressor(random_state=1515706236),
 DecisionTreeRegressor(random_state=484695009),
 DecisionTreeRegressor(random_state=1869768397)]

In [35]:
bag_model.estimators_samples_

[array([ 34,   9, 121, 111, 134, 147, 166, 136, 173, 140,  93,  92,  15,
        153,  89,  38,  81,  92,  89, 109, 140,  96,  12,  69, 183,  11,
         84, 124, 100,  62, 128, 172,  56, 115,   6, 192, 184,  57, 157,
         51, 112,  79, 143, 172,  72, 127, 150,   3,  43, 128, 116,  59,
         60, 195, 180,  25,  32,  70, 192,  21,  83, 105, 154,  39,  39,
         43,  42,  39,  41,  42,  47, 180,  69, 179, 181,   6, 125, 110,
        120, 120, 165, 178,  33, 139, 112,  26, 122, 141, 166, 122, 181,
         61, 175,  74, 172, 118,  56, 156, 181, 182, 171, 173, 162,  44,
         11, 140, 132, 167,  16, 186, 104,  84,   1, 143,  75, 173, 118,
        112,  98,  99, 153,  15,   5,   5,  97, 104, 192, 167, 108, 175,
        150,  44, 124,  72,  27, 130,  18,  16,  25,  48, 117, 125,  87,
        134,  49, 174,  21,  23, 159, 132, 105, 173,  66,  48,  91, 141,
         20, 158,  14, 108, 146, 121,  62, 163,  49, 175, 125, 186, 176,
        165, 182,  35,  44,  52,  46,  39,  84,  47

In [36]:
bag_model.estimators_features_

[array([ 9, 13,  6,  5,  1,  0, 15, 12, 12, 15, 18,  6, 11,  0, 16,  6, 10,
         6,  4]),
 array([16,  5, 16,  7,  1, 15,  4,  6, 12,  9,  5,  2, 12, 18, 11, 15, 11,
        10, 12]),
 array([ 2,  3,  6,  7,  7, 10,  8,  2,  9, 16,  7,  2, 15, 16,  8,  2, 15,
        11,  2]),
 array([11,  0, 10, 18,  0,  5, 17, 13,  5, 18, 15, 14, 14,  4,  9,  6, 15,
        18, 13]),
 array([ 3,  7, 12,  9, 18, 14,  2,  6,  4,  6,  6,  6,  1, 12, 18,  9,  7,
        14, 13]),
 array([ 3,  6, 18,  4,  8, 18, 14, 17,  8,  5, 10, 17,  1, 17, 14, 18, 13,
         3,  0]),
 array([ 5,  4,  0,  4,  9, 15,  7,  6, 15, 10, 10,  7, 12,  6, 10,  2, 16,
        15,  9]),
 array([ 8,  1,  9, 18,  0, 14,  9, 18,  2, 13,  5,  7,  0,  8,  8, 14,  3,
         0,  5]),
 array([ 1,  5,  8,  4,  0, 11, 16, 17, 10,  1, 14,  5, 13,  0, 15,  2,  0,
        10, 15]),
 array([13,  3,  9,  7, 12, 13,  7, 14, 13, 13, 16,  8,  4, 14, 15, 14, 14,
        14,  8])]

# Tahmin

In [37]:
y_pred = bag_model.predict(x_test)

In [38]:
np.sqrt(mean_squared_error(y_test,y_pred))

341.97457821983375

In [39]:
iki_y_pred = bag_model.estimators_[1].fit(x_train,y_train).predict(x_test)

In [40]:
np.sqrt(mean_squared_error(y_test,iki_y_pred
                          ))

529.646813101615

# Model Tuning

In [41]:
bag_params = {"n_estimators": range(2,100)}

In [42]:
bag_cv_model = GridSearchCV(bag_model,bag_params,cv=10)

In [43]:
bag_cv_model.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=BaggingRegressor(bootstrap_features=True),
             param_grid={'n_estimators': range(2, 100)})

In [44]:
bag_tuned = BaggingRegressor(n_estimators=14, random_state=45)

In [45]:
bag_tuned.fit(x_train,y_train)

BaggingRegressor(n_estimators=14, random_state=45)

In [47]:
y_pred = bag_tuned.predict(x_test)

In [48]:
np.sqrt(mean_squared_error(y_test,y_pred))

346.457987188104